# Bank Renege
Covers:

Resources: Resource

Condition events

A counter with a random service time and customers who renege. Based on the program bank08.py from TheBank tutorial of SimPy 2. (KGM)

This example models a bank counter and customers arriving at random times. Each customer has a certain patience. She waits to get to the counter until she’s at the end of her tether. If she gets to the counter, she uses it for a while before releasing it.

New customers are created by the source process every few time steps.

In [1]:
import numpy as np

np.exp(0.2)

1.2214027581601699

In [2]:
"""
Bank renege example

Covers:

- Resources: Resource
- Condition events

Scenario:
  A counter with a random service time and customers who renege. Based on the
  program bank08.py from TheBank tutorial of SimPy 2. (KGM)

"""

import random

import simpy

RANDOM_SEED = 42
NEW_CUSTOMERS = 5  # Total number of customers
INTERVAL_CUSTOMERS = 10.0  # Generate new customers roughly every x seconds
MIN_PATIENCE = 1  # Min. customer patience
MAX_PATIENCE = 3  # Max. customer patience


def source(env, number, interval, counter):
    """Source generates customers randomly"""
    for i in range(number):
        c = customer(env, f'Customer{i:02d}', counter, time_in_bank=12.0)
        env.process(c)
        #print(f'Customer{i:02d}',"AAA")
        
        interval = 1
        t = random.expovariate(1.0 / interval)
        print(f'Customer{i:02d}', "t", t)
        yield env.timeout(t)
        print(f'Customer{i:02d}', "CCC")        


def customer(env, name, counter, time_in_bank):
    """Customer arrives, is served and leaves."""
    arrive = env.now
    print(f'{arrive:7.4f} {name}: Here I am')
    
    with counter.request() as req:
        patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        patience= 10
        # Wait for the counter or abort at the end of our tether
        results = yield req | env.timeout(patience)
        print(f'{env.now:7.4f} {name} results',results)
        wait = env.now - arrive

        if req in results:
            # We got to the counter
            print(f'{env.now:7.4f} {name}: Waited {wait:6.3f}')

            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)
            print(f'{env.now:7.4f} {name}: Finished')

        else:
            # We reneged
            print(f'{env.now:7.4f} {name}: RENEGED after {wait:6.3f}')

    
# Setup and start the simulation
#print('Bank renege')
random.seed(RANDOM_SEED)
env = simpy.Environment()

# Start processes and run
counter = simpy.Resource(env, capacity=1)
env.process(source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter))
env.run()

Customer00 t 1.020060287274801
 0.0000 Customer00: Here I am
 0.0000 Customer00 results <ConditionValue {<Request() object at 0x2bf69d23790>: None}>
 0.0000 Customer00: Waited  0.000
Customer00 CCC
Customer01 t 0.25258618567011354
 1.0201 Customer01: Here I am
Customer01 CCC
Customer02 t 1.1291730087985927
 1.2726 Customer02: Here I am
Customer02 CCC
Customer03 t 0.09095240460656615
 2.4018 Customer03: Here I am
Customer03 CCC
Customer04 t 0.030250177207358588
 2.4928 Customer04: Here I am
Customer04 CCC
 3.8595 Customer00: Finished
 3.8595 Customer01 results <ConditionValue {<Request() object at 0x2bf6a160050>: None}>
 3.8595 Customer01: Waited  2.839
11.2726 Customer02 results <ConditionValue {<Timeout(10) object at 0x2bf6a223c10>: None}>
11.2726 Customer02: RENEGED after 10.000
12.3065 Customer01: Finished
12.3065 Customer03 results <ConditionValue {<Request() object at 0x2bf6a223fd0>: None}>
12.3065 Customer03: Waited  9.905
12.4928 Customer04 results <ConditionValue {<Timeout(10) 